In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets, models

In [18]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [19]:
# Kaggle Kernel-dependent paths
train_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
val_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_dir = '/kaggle/input/new-plant-diseases-dataset/test'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [20]:

# Number of classes
num_classes = len(train_dataset.classes)
num_classes

38

In [21]:
# Load pre-trained ResNet model
model = models.resnet34(pretrained=True)


In [22]:
# Fine-tuning: Unfreeze all layers
for param in model.parameters():
    param.requires_grad = True


In [23]:
# Replace the final classification layer with a new one
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)


In [24]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [10]:
# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model (Fine-tuning)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Fine-tuning - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


Fine-tuning - Epoch 1/10, Loss: 0.4009
Fine-tuning - Epoch 2/10, Loss: 0.0433
Fine-tuning - Epoch 3/10, Loss: 0.0206
Fine-tuning - Epoch 4/10, Loss: 0.0125
Fine-tuning - Epoch 5/10, Loss: 0.0084
Fine-tuning - Epoch 6/10, Loss: 0.0059
Fine-tuning - Epoch 7/10, Loss: 0.0046
Fine-tuning - Epoch 8/10, Loss: 0.0037
Fine-tuning - Epoch 9/10, Loss: 0.0030
Fine-tuning - Epoch 10/10, Loss: 0.0026


In [11]:
# Evaluate the model (Fine-tuning)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_ft = correct / total
print(f"Fine-tuning - Validation Accuracy: {accuracy_ft:.4f}")


Fine-tuning - Validation Accuracy: 0.9965


In [25]:
# Feature Extraction: Freeze convolutional layers
model_fe = models.resnet34(pretrained=True)
for param in model_fe.parameters():
    param.requires_grad = False

In [26]:
# Replace the final classification layer with a new one
num_features_fe = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_features_fe, num_classes)


In [27]:
# Define loss function and optimizer
criterion_fe = nn.CrossEntropyLoss()
optimizer_fe = optim.SGD(model_fe.fc.parameters(), lr=0.001, momentum=0.9)


In [29]:
# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Move model to GPU if available
model_fe.to(device)

num_epochs = 10
# Train the model (Feature Extraction)
for epoch in range(num_epochs):
    model_fe.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_fe.zero_grad()
        outputs_fe = model_fe(inputs)
        loss_fe = criterion_fe(outputs_fe, labels)
        loss_fe.backward()
        optimizer_fe.step()
        running_loss += loss_fe.item() * inputs.size(0)

    epoch_loss_fe = running_loss / len(train_loader.dataset)
    print(f"Feature Extraction - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss_fe:.4f}")


Feature Extraction - Epoch 1/10, Loss: 1.2186
Feature Extraction - Epoch 2/10, Loss: 0.4694
Feature Extraction - Epoch 3/10, Loss: 0.3577
Feature Extraction - Epoch 4/10, Loss: 0.3048
Feature Extraction - Epoch 5/10, Loss: 0.2718
Feature Extraction - Epoch 6/10, Loss: 0.2488
Feature Extraction - Epoch 7/10, Loss: 0.2318
Feature Extraction - Epoch 8/10, Loss: 0.2183
Feature Extraction - Epoch 9/10, Loss: 0.2081
Feature Extraction - Epoch 10/10, Loss: 0.1998


In [30]:
# Evaluate the model (Feature Extraction)
model_fe.eval()
correct_fe = 0
total_fe = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs_fe = model_fe(inputs)
        _, predicted_fe = torch.max(outputs_fe, 1)
        total_fe += labels.size(0)
        correct_fe += (predicted_fe == labels).sum().item()

accuracy_fe = correct_fe / total_fe
print(f"Feature Extraction - Validation Accuracy: {accuracy_fe:.4f}")


Feature Extraction - Validation Accuracy: 0.9468
